In [ ]:
import numpy
from matplotlib import pyplot
%matplotlib inline

from matplotlib import rcParams
rcParams['font.family'] = 'serif'
rcParams['font.size'] = 14

## General spring-mass system

The simplest mechanical oscillating system is a mass $m$ attached to a spring, without friction. We discussed this system in the [previous lesson](http://go.gwu.edu/engcomp3lesson3). In general, though, these systems are subject to friction—represented by a mechanical damper—and a driving force. Also, the spring's restoring force could be a nonlinear function of position, $k(x)$.

<img src="images/damped-spring.png" style="width: 500px;"/> 
#### General spring-mass system, with driving and damping.

Newton's law applied to the general (driven, damped, nonlinear) spring-mass system is:

\begin{equation}
 m \ddot{x} = F(t) -b(\dot{x}) - k(x)
\end{equation}

where
* $F(t)$ is the driving force
* $b(\dot{x})$ is the damping force
* $k(x)$ is the restoring force, possibly nonlinear

Written as a system of two differential equations, we have:

\begin{eqnarray}
\dot{x} &=& v, \nonumber\\
\dot{v} &=& \frac{1}{m} \left( F(t) - k(x) - f(v) \right).
\end{eqnarray}

With the state vector,
\begin{equation}
\mathbf{x} = \begin{bmatrix}
x \\ v
\end{bmatrix},
\end{equation}

the differential equation in vector form is:

\begin{equation}
\dot{\mathbf{x}} = \begin{bmatrix}
v \\ \frac{1}{m} \left( F(t) - k(x) - f(v) \right)
\end{bmatrix}.
\end{equation}

In this more general system, the time variable could appear explicitly on the right-hand side, via the driving function $F(t)$. We'll need to adapt the code for the time-stepping function to take the time as an additional argument. 

For example, the `euler_cromer()` function we defined in the previous lesson took three arguments: `state, rhs, dt`—the state vector, the Python function computing the right-hand side of the differential equation, and the time step. Let's re-work that function now to take an additional `time` variable, which also gets used in the `rhs` function.

In [ ]:
# new version of the function, taking time as explicit argument
def euler_cromer(state, rhs, time, dt):
    '''Update a state to the next time increment using Euler-Cromer's method.
    
    Arguments
    ---------
    state : state vector of dependent variables
    rhs   : function that computes the RHS of the DE, taking (state, time)
    t     : float, time instant
    dt    : float, time step
    
    Returns
    -------
    next_state : state vector updated after one time increment'''
    
    mid_state = state + rhs(state, time)*dt # Euler step
    mid_derivs = rhs(mid_state, time)       # update derivatives
    
    next_state = numpy.array([mid_state[0], state[1] + mid_derivs[1]*dt])
    
    return next_state

### Case with linear damping

Let's look at the behavior of a system with linear restoring force, linear damping, but no driving force: $k(x)= kx$, $f(v)=bv$, $F(t)=0$.


\begin{equation}
\dot{\mathbf{x}} = \begin{bmatrix}
v \\ \frac{1}{m} \left( - kx - bv \right)
\end{bmatrix}.
\end{equation}

Even though this system does not explicitly use the time variable in the right-hand side, we still include `time` as an argument to the function, so that it is consistent with our new design for the `euler_cromer()` step. We include `time` in the arguments list, but it is not used inside the function code. We also specify a _default value_ for this argument by writing `time=0` in the arguments list: that allows us to also call the function leaving the `time` argument blank, if we wanted to (in which case, it will automatically be assigned its default value of 0).

In [ ]:
def dampedspring(state, time=0):
    '''Computes the right-hand side of the spring-mass differential 
    equation, with linear damping.
    
    Arguments
    --------- 
    state : state vector of two dependent variables
    time : float, time instant
    
    Returns 
    -------
    derivs: derivatives of the state vector
    '''
    
    derivs = numpy.array([state[1], 1/m*(-k*state[0]-b*state[1])])
    return derivs

The following example is from section 4.3.9 of Ref. [1]. 
https://link.springer.com/chapter/10.1007%2F978-3-319-32428-9_4#Fig25

In [ ]:
m = 1
k = 1
b = 0.3

In [ ]:
x0 = 1    # initial position
v0 = 0    # initial velocity

In [ ]:
T = 12*numpy.pi
N = 5000
dt = T/N

t = numpy.linspace(0, T, N)

In [ ]:
num_sol = numpy.zeros([N,2]) #initialize solution array

#Set intial conditions
num_sol[0,0] = x0
num_sol[0,1] = v0

In [ ]:
for i in range(N-1):
    num_sol[i+1] = euler_cromer(num_sol[i], dampedspring, t[i], dt)

In [ ]:
fig = pyplot.figure(figsize=(6,4))

pyplot.plot(t, num_sol[:, 0], linewidth=2, linestyle='-')
pyplot.xlabel('Time [s]')
pyplot.ylabel('$x$ [m]')
pyplot.title('Damped spring-mass system with Euler-Cromer method.\n')
pyplot.figtext(0,0,'$m=1$, $k=1$, $b=0.3$');

The result above shows that the oscillations die down over a few periods. Matches [Fig. 4.27](https://link.springer.com/chapter/10.1007%2F978-3-319-32428-9_4#Fig27) of Ref. [1].

In [ ]:
u'\u03C9'

In [ ]:
A = 0.5
ω = 3

In [ ]:
from math import sin
F = lambda time: A*sin(ω*time)

Unicode for greek letters from https://gist.github.com/beniwohli/765262

In [ ]:
def drivenspring(state, time):
    '''Computes the right-hand side of the spring-mass differential 
    equation, with sinusoidal driving and linear damping.
    
    Arguments
    ---------   
    state : array of two dependent variables [x, v]^T
    time  : float, time instant
    
    Returns 
    -------
    derivs: array of two derivatives [v, 1/m*(F(t)-kx -bv)]^T
    '''
      
    derivs = numpy.array([state[1], 1/m*(F(time)-k*state[0]-b*state[1])])
    return derivs

In [ ]:
for i in range(N-1):
    num_sol[i+1] = euler_cromer(num_sol[i], drivenspring, t[i], dt)

In [ ]:
fig = pyplot.figure(figsize=(6,4))

pyplot.plot(t, num_sol[:, 0], linewidth=2, linestyle='-')
pyplot.xlabel('Time [s]')
pyplot.ylabel('$x$ [m]')
pyplot.title('Damped spring-mass system with Euler-Cromer method.\n');

In [ ]:
ω = 1
b = 0.1

In [ ]:
for i in range(N-1):
    num_sol[i+1] = euler_cromer(num_sol[i], drivenspring, t[i], dt)

In [ ]:
fig = pyplot.figure(figsize=(6,4))

pyplot.plot(t, num_sol[:, 0], linewidth=2, linestyle='-')
pyplot.xlabel('Time [s]')
pyplot.ylabel('$x$ [m]')
pyplot.title('Driven spring-mass system with Euler-Cromer method.\n');

In [ ]:
numpy.max(num_sol[:,0])

Ideas from SciPy tutorial to plot direction field
http://scipy-cookbook.readthedocs.io/items/LoktaVolterraTutorial.html

In [ ]:
m = 1
k = 1
b = 0.3

x0 = 3    # initial position
v0 = 3    # initial velocity

T = 12*numpy.pi
N = 5000
dt = T/N

t = numpy.linspace(0, T, N)
num_sol = numpy.zeros([N,2]) #initialize solution array

#Set intial conditions
num_sol[0,0] = x0
num_sol[0,1] = v0

for i in range(N-1):
    num_sol[i+1] = euler_cromer(num_sol[i], dampedspring, t[i], dt)

In [ ]:
coords = numpy.linspace(-4,4,11)
X, Xdot = numpy.meshgrid(coords, coords)

In [ ]:
F1, F2 = dampedspring([X,Xdot])

In [ ]:
M = numpy.hypot(F1,F2)
M[ M == 0] = 1
F1 = F1/M
F2 = F2/M
fig = pyplot.figure(figsize=(7,7))
pyplot.quiver(X,Xdot, F1,F2, pivot='mid', alpha=0.5)
pyplot.plot(num_sol[:,0], num_sol[:,1], color= '#0096d6', linewidth=2)
pyplot.figtext(0,0,'$m=1$, $k=1$, $b=0.3$');